## 文档的创建和实例

In [1]:
from mongoengine import connect, disconnect
from mongoengine import Document
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD

In [2]:
# 连接到已创建的数据库
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)
connect(TEST_DB2, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB2, alias=TEST_DB2)
connect(TEST_DB3, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB3, alias=TEST_DB3)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### 静态文档和动态文档

In [3]:
# 继承自Document的文档
class Book1(Document):
    bookid = StringField()
    name = StringField()
    count = IntField()
    meta = {"db_alias": TEST_DB1}
Book1(bookid="wx000006", name="水浒传", count=29).save()

In [6]:
# 动态文档, 可以在原有的数据模式上动态添加字段
from mongoengine import DynamicDocument
class BookDynamic(DynamicDocument):
    bookid = StringField()
    meta = {"db_alias": TEST_DB1}
bookd = BookDynamic(bookid="wx09872")
bookd.tag = ["小说", "经典著作"]
bookd.save()

<BookDynamic: BookDynamic object>

### 一些主要并常用的字段类型

In [13]:
from mongoengine import BooleanField, DateTimeField, DictField, EmailField, FloatField, ListField, ReferenceField

In [61]:
class BookType(Document):
    type_name = StringField()
    meta = {"db_alias": TEST_DB1}

In [17]:
BookType(type_name="经济类").save()
BookType(type_name="文学类").save()
BookType(type_name="计算机").save()

<BookType: BookType object>

In [59]:
class BookAuthor(Document):
    author_name = StringField()
    meta = {"db_alias": TEST_DB1}

In [62]:
class BookAllField(Document):
    bookid = StringField()
    name = StringField()
    ifnewbook = BooleanField()  # 是否为新上架图书
    shelves_date = DateTimeField()
    author = ReferenceField(BookAuthor)  # 使引用字段执行其他集合
    author_email = EmailField()
    price = FloatField()
    book_type = ReferenceField(BookType)
    tag = ListField()
    meta = {"db_alias": TEST_DB1}

In [29]:
book_all = BookAllField(bookid="wx092761")

In [30]:
# 包含使用引用字段的情形
book_all.name = "史记"
book_all.ifnewbook = False
book_all.author = BookAuthor(author_name="司马迁").save()  # 此时的引用可以成功
book_all.price = 19.8
# book_all.book_type = BookType(type_name="文学类")  # 此时不能引用，需要有保存动作，或者查询出已经保存的项目
# book_all.book_type = BookType.objects(type_name="文学类") # 没保存成功
book_all.tag = ["历史", "经典"]
book_all.save()
# shelves_date， author_email， ？？

<BookAllField: BookAllField object>

### 使用Enum, choice, 限定某一个字段值的范围

Values are stored underneath as is, so it will only work with **simple types (str, int, etc)** that are bson encodable

In [43]:
from enum import Enum  # Python 3.4 中新增加了 Enum 枚举类。也就是说，对于这些实例化对象个数固定的类，可以用枚举类来定义
class BookTypeEnum(Enum):
    IT = "计算机"
    ECONOMIC = "经济"
    LITERATURE = "文学"
    HISTORY = "历史"
    MATH = "数学"

In [46]:
from mongoengine import EnumField
class BookEnum(Document):
    bookid = StringField()
    name = StringField()
    book_type = EnumField(BookTypeEnum, default=BookTypeEnum.IT)
    meta = {"db_alias": TEST_DB1}

In [36]:
# 如果赋值字段不在 Enum类 范围内，将报错
book_enum = BookEnum(bookid="wx9387221")
book_enum.name = "论语"
book_enum.book_type = BookTypeEnum.LITERATURE  # 使用Enum类赋值
book_enum.save()

<BookEnum: BookEnum object>

In [42]:
# 也可以直接使用Enum的值赋值给字段
BookEnum(bookid="wx9387e492", name="春秋", book_type="历史").save()

<BookEnum: BookEnum object>

In [45]:
# 使用字段通用参数choice的情形
class BookEnumChoice(Document):
    bookid = StringField()
    name = StringField()
    book_type = EnumField(BookTypeEnum, default=BookTypeEnum.IT, choices=[BookTypeEnum.IT, BookTypeEnum.MATH])
    meta = {"db_alias": TEST_DB1}

In [ ]:
# choices限定后，如果值不在之内，则报错 ValidationError
BookEnumChoice(bookid="wx9dd3492", name="春秋", book_type="历史").save()

In [48]:
BookEnumChoice(bookid="wx9dd3492", name="C语言基础").save()

<BookEnumChoice: BookEnumChoice object>

In [49]:
BookEnumChoice(bookid="wx9dd3492", name="Python入门经典", book_type=BookTypeEnum.MATH).save()

<BookEnumChoice: BookEnumChoice object>

### DateTimeField字段

In [53]:
from datetime import datetime
class BookDate(Document):
    bookid = StringField()
    name = StringField()
    chapter = StringField(required=True, default="2.3-2.4")
    shelves_date = DateTimeField(default=datetime.now)  # 注意，此时传入参数并不立即执行，即不添加()
    meta = {"db_alias": TEST_DB1}

In [52]:
BookDate(bookid="wx93kkid3", name="格林童话").save()  # 使用默认的日期

<BookDate: BookDate object>

In [54]:
BookDate(bookid="wx9349id3", name="小王子").save()  # 使用默认的日期

<BookDate: BookDate object>

In [56]:
# 字符串类型的日期，转化为时间类型的日期并存储至 DateTimeField 字段中
book_d = BookDate(bookid="wx9326700", name="道德经")
book_d.shelves_date = datetime.strptime("2020/12/29 8:8:00",'%Y/%m/%d %H:%M:%S')
book_d.save()
# 使用 datetime 进行日期数据类型的操作

<BookDate: BookDate object>

### 关于修改文档类:
前期建立的文档类(静态),并已经建立并使用，之后修改新增字段时，前期使用生成的文档字段中针对后期新增的部分为空值。
在原有基础上操作修改后的文档类, 此时并不会报错。